## Predicting Credit Fraud
Dealing with data Imbalance

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

Get the data at https://www.kaggle.com/mlg-ulb/creditcardfraud

In [3]:
data = pd.read_csv("data/creditcard.csv")
data.shape

(284807, 31)

In [52]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [53]:
# Percent of positive samples
100*data.Class.sum()/284807

0.1727485630620034

## Train and validation split

In [54]:
X = data.iloc[:,:-1]
Y = data.Class.values
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

In [55]:
X_train.shape

(227845, 30)

In [20]:
def get_scaled_dataset():
    x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    return x_train, x_val, y_train, y_val 

## Logistic regression baseline

In [21]:
x_train, x_val,  y_train, y_val = get_scaled_dataset()

In [22]:
y_train.shape, y_val.shape

((227845,), (56962,))

In [23]:
lr = LogisticRegression(random_state=3)
lr.fit(x_train, y_train)

LogisticRegression(random_state=3)

In [24]:
y_hat_val = lr.predict_proba(x_val)[:, 1]

In [25]:
precision, recall, thresholds = precision_recall_curve(y_val, y_hat_val)
auc_precision_recall = auc(recall, precision)
auc_precision_recall

0.7427196002196462

In [26]:
roc_auc_score(y_val, y_hat_val)

0.9560270189955554

## Logistic regression with weights

In [27]:
for w in [2, 5, 10, 100, 200]:
    weights = {0:1.0, 1:w}
    lr = LogisticRegression(solver='lbfgs', class_weight=weights)
    lr.fit(x_train, y_train)
    y_hat_val = lr.predict_proba(x_val)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, y_hat_val)
    auc_precision_recall = auc(recall, precision)
    auc_roc = roc_auc_score(y_val, y_hat_val)
    print("weight: %d auc precision recall %.3f auc roc %.3f" % (w, auc_precision_recall, auc_roc ))

weight: 2 auc precision recall 0.747 auc roc 0.962
weight: 5 auc precision recall 0.750 auc roc 0.965
weight: 10 auc precision recall 0.748 auc roc 0.967
weight: 100 auc precision recall 0.744 auc roc 0.970
weight: 200 auc precision recall 0.742 auc roc 0.971


## Smote
`pip install imblearn`

In [45]:
from imblearn.over_sampling import SMOTE

In [16]:
x_train, x_val,  y_train, y_val = get_scaled_dataset()

In [17]:
oversample = SMOTE(sampling_strategy=0.5)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [18]:
lr = LogisticRegression(random_state=3)
lr.fit(x_train, y_train)

LogisticRegression(random_state=3)

In [19]:
y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, y_hat_val)
auc_precision_recall = auc(recall, precision)
auc_roc = roc_auc_score(y_val, y_hat_val)
print("auc precision recall %.3f auc roc %.3f" % (auc_precision_recall, auc_roc ))

auc precision recall 0.753 auc roc 0.970


## Undersample with One Sided Selection
This is too slow

In [ ]:
from imblearn.under_sampling import OneSidedSelection, RandomUnderSampler
import time

In [ ]:
x_train, x_val,  y_train, y_val = get_scaled_dataset()

In [ ]:
x_train.shape

In [ ]:
undersample = RandomUnderSampler(sampling_strategy=0.02)
x_train, y_train = undersample.fit_resample(x_train, y_train)

In [ ]:
x_train.shape

In [ ]:
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=10, n_jobs=-1)
%time x_train, y_train = undersample.fit_resample(x_train, y_train)

In [ ]:
lr = LogisticRegression(random_state=3)
lr.fit(x_train, y_train)
y_hat_val = lr.predict_proba(x_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, y_hat_val)
auc_precision_recall = auc(recall, precision)
auc_roc = roc_auc_score(y_val, y_hat_val)
print("auc precision recall %.3f auc roc %.3f" % (auc_precision_recall, auc_roc ))

## Lab
1. Write a Smote pipeline with cross-validation. Note that oversampling and underssampling needs to be done in the cross-validation loop.

In [32]:
X = data.iloc[:,:-1]
Y = data.Class.values
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

In [33]:
X_train.reset_index(drop=True, inplace=True)

In [34]:
from sklearn.model_selection import StratifiedKFold

In [40]:
def smote_CV(X, Y, kf, sampling_strategy=0.5):
    pred = []
    ys = []
    for train_index, test_index in kf.split(X, Y):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = Y[train_index], Y[test_index]
        scaler = StandardScaler() # creates the scaler
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
        oversample = SMOTE(sampling_strategy=sampling_strategy)
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        lr = LogisticRegression(random_state=3).fit(X_train, y_train)
        pred.append(lr.predict_proba(X_val)[:, 1])
        ys.append(y_val)
    
    ys = np.concatenate(ys)
    pred = np.concatenate(pred)
    precision, recall, thresholds = precision_recall_curve(ys, pred)
    auc_precision_recall = auc(recall, precision)
    auc_roc = roc_auc_score(ys, pred)
    return auc_precision_recall, auc_roc

In [41]:
X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,161919.0,1.946747,-0.752526,-1.355130,-0.661630,1.502822,4.024933,-1.479661,1.139880,1.406819,...,-0.134435,0.076197,0.297537,0.307915,0.690980,-0.350316,-0.388907,0.077641,-0.032248,7.32
1,124477.0,2.035149,-0.048880,-3.058693,0.247945,2.943487,3.298697,-0.002192,0.674782,0.045826,...,-0.227279,0.038628,0.228197,0.035542,0.707090,0.512885,-0.471198,0.002520,-0.069002,2.99
2,41191.0,-0.991920,0.603193,0.711976,-0.992425,-0.825838,1.956261,-2.212603,-5.037523,0.000772,...,1.280856,-2.798352,0.109526,-0.436530,-0.932803,0.826684,0.913773,0.038049,0.185340,175.10
3,132624.0,2.285718,-1.500239,-0.747565,-1.668119,-1.394143,-0.350339,-1.427984,0.010010,-1.118447,...,-0.490642,-0.139670,0.077013,0.208310,-0.538236,-0.278032,-0.162068,0.018045,-0.063005,6.10
4,59359.0,-0.448747,-1.011440,0.115903,-3.454854,0.715771,-0.147490,0.504347,-0.113817,-0.044782,...,-0.275297,-0.243245,-0.173298,-0.006692,-1.362383,-0.292234,-0.144622,-0.032580,-0.064194,86.10


In [42]:
skf = StratifiedKFold(n_splits=3)
skf.get_n_splits(X_train, y_train)

3

In [43]:
X_train.shape, y_train.shape

((227845, 30), (227845,))

In [47]:
for s in [0.05, 0.1, 0.2, 0.5]:
    auc_precision_recall, auc_roc = smote_CV(X_train.values, y_train, skf, sampling_strategy=s)
    print("sampling_strategy: %.3f auc precision recall %.3f auc roc %.3f" % (s, auc_precision_recall, auc_roc ))

sampling_strategy: 0.050 auc precision recall 0.764 auc roc 0.979
sampling_strategy: 0.100 auc precision recall 0.761 auc roc 0.979
sampling_strategy: 0.200 auc precision recall 0.756 auc roc 0.979
sampling_strategy: 0.500 auc precision recall 0.761 auc roc 0.979


## References
1. https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets
2. https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/